In [4]:
import requests
import pandas as pd
from datetime import datetime

In [5]:
target_account = "cosmos152fyy23gpnxamgn4u6jqmjfnfq344cyen8xjjs"

In [6]:

df = pd.DataFrame(columns=['time','tx_hash','tx_type',
'coin','token_type','amount','from_address','to_address'])

In [53]:
#https://atomscan.com/accounts/cosmos152fyy23gpnxamgn4u6jqmjfnfq344cyen8xjjs 
#영원히 안녕...

send_url = f"https://cosmos.lcd.atomscan.com/cosmos/tx/v1beta1/txs?pagination.limit=20&pagination.offset=0&orderBy=ORDER_BY_DESC&events=transfer.sender%3D%27{target_account}%27"
send_res = requests.get(send_url)
send_json_data = send_res.json()

# sender reciever가 둘 다 나와서, 시간으로 하나봐요
# https://cosmos.lcd.atomscan.com/cosmos/tx/v1beta1/txs?pagination.limit=10&pagination.offset=0&orderBy=ORDER_BY_DESC&events=transfer.sender%3D%27cosmos152fyy23gpnxamgn4u6jqmjfnfq344cyen8xjjs%27
# https://cosmos.lcd.atomscan.com/cosmos/tx/v1beta1/txs?pagination.limit=10&pagination.offset=0&orderBy=ORDER_BY_DESC&events=transfer.recipient%3D%27cosmos152fyy23gpnxamgn4u6jqmjfnfq344cyen8xjjs%27

In [58]:
# print(json_data.keys())
# total은 limit에 관련 없이 전체 15개를 말합니다
num_of_total_send_tx = int(send_json_data['pagination']['total'])
send_txs = send_json_data['txs']
len(send_txs)
print(send_txs[0])

{'body': {'messages': [{'@type': '/cosmos.bank.v1beta1.MsgSend', 'from_address': 'cosmos152fyy23gpnxamgn4u6jqmjfnfq344cyen8xjjs', 'to_address': 'cosmos1j8pp7zvcu9z8vd882m284j29fn2dszh05cqvf9', 'amount': [{'denom': 'uatom', 'amount': '141627306'}]}], 'memo': '101804568', 'timeout_height': '0', 'extension_options': [], 'non_critical_extension_options': []}, 'auth_info': {'signer_infos': [{'public_key': {'@type': '/cosmos.crypto.secp256k1.PubKey', 'key': 'A7II3RdLG4dc3BTM13VSnlfoa6C5IzADYNNdjes/5ZEQ'}, 'mode_info': {'single': {'mode': 'SIGN_MODE_LEGACY_AMINO_JSON'}}, 'sequence': '14'}], 'fee': {'amount': [{'denom': 'uatom', 'amount': '2000'}], 'gas_limit': '80000', 'payer': '', 'granter': ''}}, 'signatures': ['YbWzs+AqRzV6So21haPNgRA7kpqpyvbEs5/VLGTCGV1ic/cre2oF8HHL0Lbi8mo/m0kLSq1CvElouCP3e44dyQ==']}


In [ ]:
# 끝이 0일때까지 request를 보내다가 안되면 취소
# limit은 20으로 고정, offset을 변화시키면서?

In [59]:
number_of_iter = ( num_of_total_send_tx // 20 ) + 1
number_of_iter

1

In [210]:

for tx in send_txs:
    tx_time_stamp = datetime.fromtimestamp(tx['timestamp']/1000)
    tx_hash = tx['hash']
    tx_result = tx['result']
    if tx_result != 'SUCCESS':
        continue
    tx_coin = tx['tokenInfo']['tokenName']
    tx_token_decimal = tx['tokenInfo']['tokenDecimal']
    tx_amount = int(tx['amount']) / tx_token_decimal
    tx_from_address = tx['ownerAddress']
    tx_to_address = tx['toAddress']
    tx_token_type = tx['tokenType']
    try:
        tx_type = tx['trigger_info']['methodName']
    except:
        tx_type = "Transfer TRX"
    df = pd.concat([df, pd.DataFrame([[tx_time_stamp, tx_hash,tx_type, 
    tx_coin, tx_token_type,tx_amount,tx_from_address ,
    tx_to_address]], 
    columns=['time','tx_hash','tx_type',
'coin','token_type','amount','from_address','to_address'])])
    



In [212]:
for i in range(1, number_of_iter):

    start = 20 * i
    
    url = f"https://apilist.tronscan.org/api/transaction?sort=-timestamp&count=true&limit=20&start={start}&address={target_account}"
    res = requests.get(url)
    json_data = res.json()
    # tx를 안받아왔다
    txs = json_data['data']
    for tx in txs:
        tx_time_stamp = datetime.fromtimestamp(tx['timestamp']/1000)
        tx_hash = tx['hash']
        tx_result = tx['result']
        if tx_result != 'SUCCESS':
            continue
        tx_coin = tx['tokenInfo']['tokenName']
        tx_token_decimal = tx['tokenInfo']['tokenDecimal']
        tx_amount = int(tx['amount']) / tx_token_decimal
        tx_from_address = tx['ownerAddress']
        tx_to_address = tx['toAddress']
        tx_token_type = tx['tokenType']
        try:
            tx_type = tx['trigger_info']['methodName']
        except:
            tx_type = "Transfer TRX"
        # df를 합치는 데에서 오류가 있는 거 같다.
        df = pd.concat([df, pd.DataFrame([[tx_time_stamp, tx_hash,tx_type, 
        tx_coin, tx_token_type,tx_amount,tx_from_address ,
        tx_to_address]], 
        columns=['time','tx_hash','tx_type',
    'coin','token_type','amount','from_address','to_address'])])
    # 뭐가 틀렸을까...?    


    


https://apilist.tronscan.org/api/transaction?sort=-timestamp&count=true&limit=20&start=20&address=TKqUVHFV4ayYZR138m2BiVPktn9XZXLZ7W
https://apilist.tronscan.org/api/transaction?sort=-timestamp&count=true&limit=20&start=40&address=TKqUVHFV4ayYZR138m2BiVPktn9XZXLZ7W
https://apilist.tronscan.org/api/transaction?sort=-timestamp&count=true&limit=20&start=60&address=TKqUVHFV4ayYZR138m2BiVPktn9XZXLZ7W
https://apilist.tronscan.org/api/transaction?sort=-timestamp&count=true&limit=20&start=80&address=TKqUVHFV4ayYZR138m2BiVPktn9XZXLZ7W


In [216]:
file_name =  F'트론 입출금 {target_account}.xlsx'
df.to_excel(file_name, index=False)
